Upload files to colab

Translation function

In [0]:
#!/usr/bin/env python
# coding: utf-8

# Translate sentences to English and save mapping dicitionary as json.

# Based on https://github.com/ruanchaves/assin/blob/master/sources/translate.py

# Parameters:
# *   sentences =
# *   dictpath =  json dictionary file mapping original sentence with translation
# 
# 

# In[ ]:


from google.cloud import translate_v2 as translate
from os.path import isfile
from os import environ
import six
import json

def translate2dict(sentences, dictpath):
    environ['GOOGLE_APPLICATION_CREDENTIALS'] = './key.json'
    translate_client = translate.Client()

    def translation(text):
        if isinstance(text, six.binary_type):
            text = text.decode('utf-8')
        result = translate_client.translate(text,'pt-br')

        return result['translatedText']

    if not isfile(dictpath):
        with open(dictpath, 'w') as f:
            json.dump({}, f)

    for item in sentences:
        with open(dictpath) as f:
            translations = json.load(f)
        
        if item not in translations.keys():
            translations[item] = translation(item)
            
            with open(dictpath, 'w+') as f:
                json.dump(translations, f)

Prepare input

In [2]:
from pprint import pprint
import pandas as pd
import os

sentences = list()
label_idx = [
             'sentence1',
             'sentence2',
            ]
length = 0

for f in ['train.tsv', 'test.tsv', 'dev.tsv']:
    table = pd.read_csv(f, sep = '\t', quoting=3)
    print(f) 
    #print(table.head())
    #print('\n'*3)
    
    for idx in label_idx:
        label = table[idx].copy()
        print(label)
        print('\n'*2)

        sentences.extend(list(label))
        length = length + label.size


assert length == len(sentences)
sentences = set(sentences)

train.tsv
0      I stuck a pin through a carrot. When I pulled ...
1      John couldn't see the stage with Billy in fron...
2      The police arrested all of the gang members. T...
3      Steve follows Fred's example in everything. He...
4      When Tatyana reached the cabin, her mother was...
                             ...                        
630    When Tommy dropped his ice cream, Timmy giggle...
631    Mark told Pete many lies about himself, which ...
632     Tom said "Check" to Ralph as he took his bishop.
633    Emma's mother had died long ago, and her place...
634    Although they ran at about the same speed, Sue...
Name: sentence1, Length: 635, dtype: object



0                                 The carrot had a hole.
1                                      John is so short.
2      The police were trying to stop the drug trade ...
3                           Steve influences him hugely.
4      mother was careful not to disturb her, undress...
                             ..

Translate

In [0]:
translate2dict(sentences,  'wnli.json')

Replace with translation

In [0]:
from os import mkdir
import json

with open('wnli.json') as reader:
    dictionary = json.load(reader)

mkdir('translation')
for f in ['train.tsv', 'test.tsv', 'dev.tsv']:
    table = pd.read_csv(f, sep = '\t', quoting=3)
    for idx in label_idx:
        table[idx] = table[idx].map(dictionary)
    table.to_csv(f'translation/{f}', index = None, sep = '\t')